In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_validate
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

SEED = 1

2022-05-12 16:17:40.393753: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-12 16:17:40.393802: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
data = pd.read_csv('AMLtrain.csv')

In [3]:
target_col = ['respsimple']
y = data[target_col[0]]

#change output to 0, 1 class label
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)

In [4]:
cat_columns = ['Patient_id','SEX','PRIORMAL','PRIORCHEMO','PRIORXRT','Infection','cytocat','ITD','D835','RasStat','ChemoSimplest'
]

# remove categorical columns
X = data.copy().drop(columns=cat_columns).drop(columns=target_col)
for col in X.columns:
    X[col] = X[col].fillna(X[col].mean())

X.head()

,AgeatDx,AHD,WBC,ABSBLST,BMBLAST,BMMONOCYTES,BMPROM,PBBLAST,PBMONO,PBPROM,...,VHL,WTAP,XIAP,XPO1,YAP1,YAP1p,YWHAE,YWHAZ,ZNF296,ZNF346
0,63.85,0,10.8,1188.0,35.0,36.000000,2.000000,11.0,17.000000,0.000000,...,-0.722347,-0.887311,1.140104,-0.199574,-0.598099,-1.086189,-1.182815,0.336552,-0.291540,-0.156785
1,57.45,24,2.7,0.0,26.0,0.000000,0.000000,0.0,0.500000,0.000000,...,-0.401240,0.228719,2.025590,-0.268515,0.035639,-0.498447,-0.391924,-0.654588,0.110702,1.260718
2,59.49,5,1.3,0.0,35.0,1.000000,2.000000,0.0,5.000000,0.000000,...,0.322115,0.578631,-1.918884,-1.485057,-0.214276,-0.115862,0.503870,-0.263103,-0.500761,0.147560
3,46.38,0,0.8,16.0,0.0,4.162011,1.413408,2.0,11.900463,0.608333,...,0.460794,-0.724971,2.274829,0.837169,-0.240740,-0.788800,-0.986515,-0.510208,0.910813,-0.303119
4,66.95,0,2.0,160.0,34.0,3.000000,1.000000,8.0,5.000000,0.000000,...,-0.330346,-1.460989,0.004068,-1.152868,0.450986,0.753539,0.011574,-0.973533,-0.599299,0.459281


In [5]:
# split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

In [ ]:
# normalize data values
norm = MinMaxScaler().fit(X_train)

X_train = norm.transform(X_train)
X_test = norm.transform(X_test)

In [6]:
# define network model
def create_network():
    model = Sequential()
    model.add(Dense(256, input_dim=len(X.columns), activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(128, activation='relu'))
    # model.add(Dropout(0.5))
    # model.add(Dense(256, input_dim=len(X.columns), activation='relu'))
    # model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

callbacks = [EarlyStopping(monitor='val_accuracy', patience=20)]

2022-05-12 16:17:50.767410: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 16:17:50.767455: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 16:17:50.767486: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Willy): /proc/driver/nvidia/version does not exist
2022-05-12 16:17:50.767753: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# train model
n_epochs = 100
batch_size = 10

callbacks = [EarlyStopping(monitor='val_accuracy', patience=20)]

estimator = KerasClassifier(build_fn=create_network, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, callbacks=callbacks, random_state=SEED)
corss_validate(estimator, X)
# model.fit(X_train, y_train, epochs=n_epochs, batch_size=batch_size, validation_split=0.1, callbacks=callbacks)

Epoch 1/100
13/13 [==============================] - 0s 8ms/step - loss: 365.6587 - accuracy: 0.5426 - val_loss: 177.2123 - val_accuracy: 0.7333
Epoch 2/100
13/13 [==============================] - 0s 2ms/step - loss: 234.8574 - accuracy: 0.7287 - val_loss: 78.8254 - val_accuracy: 0.7333
Epoch 3/100
13/13 [==============================] - 0s 2ms/step - loss: 49.7127 - accuracy: 0.5891 - val_loss: 5.2208 - val_accuracy: 0.6000
Epoch 4/100
13/13 [==============================] - 0s 2ms/step - loss: 18.9352 - accuracy: 0.6512 - val_loss: 11.5517 - val_accuracy: 0.6000
Epoch 5/100
13/13 [==============================] - 0s 2ms/step - loss: 8.5417 - accuracy: 0.6977 - val_loss: 2.6556 - val_accuracy: 0.7333
Epoch 6/100
13/13 [==============================] - 0s 2ms/step - loss: 9.8813 - accuracy: 0.7674 - val_loss: 45.9499 - val_accuracy: 0.5333
Epoch 7/100
13/13 [==============================] - 0s 2ms/step - loss: 32.9932 - accuracy: 0.7442 - val_loss: 6.2953 - val_accuracy: 0.6000
E

In [8]:
# evaluate model across test set
score = model.evaluate(X_test, y_test, verbose=1)
print('Test accuracy:', score[1]) #accuracy of the test set

2/2 [==============================] - 0s 1ms/step - loss: 41.7695 - accuracy: 0.5946
Test accuracy: 0.5945945978164673
